# Organization of this script
### 1. Organize Data
### 2. Conduct linear mixed effect models
####     2a. Value * Attribute Type
####     2b. abs(Value) * Attribute Type

### 3. Compare to distribution from permutation testing (i.e., Step 5)
####     3a. Value * Attribute Type
####     3b. abs(Value) * Attribute Type
### 4. Plot the resulting effects.
####     4a. Value * Attribute Type
####     4b. abs(Value) * Attribute Type
### 5. Permutation testing and saving output (only run once)
####     5a. Value * Attribute Type
####     5b. abs(Value) * Attribute Type

One critical note is that permutation testing is very slow and only run once. So your first time running this script should be the following steps.

Step 1: Organize Data
Step 5: Permutation testing and saving output (only run once).

You may then proceed with steps 2, 3 and 4. Step 5 will also save its output, so in the future, you will be able to run Steps 1 through 4 without issues.

## 1. Organize Data

In [1]:
## Load initial data and modules.
%reload_ext autoreload
%autoreload 2
%run EEG_auxiliary_module_sptm_wICA.ipynb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import os.path as op
import mne
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import rle
time = np.arange(-200,1000,20)

In [1]:
## This section is dedicated to preparing the data.
## Essentially, it gets the decoding values for each color/face from each half of the experiment
## (extracted from TemporallySpecificAnalyses_SVC_mainFunctions_byBlock) and subtracts them from each other. 
## It also collects the relative values for that particular stimulus.

## Be sure to update "dataPath" and "file_name" to load the correct output.

subjects = [str(s) for s in [150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,166,167,168,169,170,171,173,174,176,177,178,180,181,182,183,185,186,188,189,190,191,192,193,194,195]]
study        = 'MADeEEG1'
studyPathInd = os.getcwd().find(study)
decoderDeltaAcc = np.empty([400,60])
decoderDeltaVal = np.empty([400,60])
subjID = np.empty([400,60])
decoderDeltaVal_permutations = np.empty([400,1000])
face_decoderDeltaVal_permutations = np.empty([200,1000])
color_decoderDeltaVal_permutations = np.empty([200,1000])
faceOrcolor= np.empty([400])


face_decoderDeltaAcc = np.empty([200,60])
face_decoderDeltaVal = np.empty([200,60])
color_decoderDeltaAcc = np.empty([200,60])
color_decoderDeltaVal = np.empty([200,60])
attr_subjID = np.empty([200,60])

for s, subj in enumerate(subjects):
    dataPath     = os.path.join('E:\Bachman\\MADeEEG1\Analysis\\Python\\EEG\\preprocessing\Results',subj,'classification_guggenmos_noJitFix')
    # Load first half of Colors
    file_name     = 'stimClassification_Colors_Stimulus_long_20ms_half1_reref.npy'
    fname    = os.path.join(dataPath,file_name)
    color_half1      = np.load(fname)
    color_half1 = color_half1[:,:,:60]


    # load second half of Colors
    file_name     = 'stimClassification_Colors_Stimulus_long_20ms_half2_reref.npy'
    fname    = os.path.join(dataPath,file_name)
    color_half2      = np.load(fname)
    color_half2 = color_half2+50 # Have to add this because it (i.e., chance level) was accidently only added to acc1 in the script.
    color_half2 = color_half2[:,:,:60]


    # Load first half of Faces
    file_name     = 'stimClassification_Faces_Stimulus_long_20ms_half1_reref.npy'
    fname    = os.path.join(dataPath,file_name)
    face_half1      = np.load(fname)
    face_half1 = face_half1[:,:,:60]


    # load second half of Faces
    file_name     = 'stimClassification_Faces_Stimulus_long_20ms_half2_reref.npy'
    fname    = os.path.join(dataPath,file_name)
    face_half2      = np.load(fname)
    face_half2 = face_half2+50 # Have to add this because it (i.e., chance level) was accidently only added to acc1 in the script.
    face_half2 = face_half2[:,:,:60]

    # Rearrange the format.
    for t in range(0,60):
        for i in range(0,5):
            for j in range(0,5):
                color_half1[j,i,t] = color_half1[i,j,t]
                color_half2[j,i,t] = color_half2[i,j,t]
                face_half1[j,i,t] = face_half1[i,j,t]
                face_half2[j,i,t] = face_half2[i,j,t]
    
        decoderDeltaAcc[(10*s):(10*s)+5,t] = np.nanmean(face_half1[:,:,t], axis = 0) - np.nanmean(face_half2[:,:,t], axis = 0)
        decoderDeltaAcc[(10*s)+5:(10*s)+10,t] = np.nanmean(color_half1[:,:,t], axis = 0) - np.nanmean(color_half2[:,:,t], axis = 0)            
        face_decoderDeltaAcc[(5*s):(5*s)+5,t] = np.nanmean(face_half1[:,:,t], axis = 0) - np.nanmean(face_half2[:,:,t], axis = 0)
        color_decoderDeltaAcc[(5*s):(5*s)+5,t] = np.nanmean(color_half1[:,:,t], axis = 0) - np.nanmean(color_half2[:,:,t], axis = 0)

    # Dummy coding attribute type
    faceOrcolor[(10*s):(10*s)+5] = [-1] * 5
    faceOrcolor[(10*s)+5:(10*s)+10] = [1] * 5
    
    # Get the face/color value
    behavPath = op.join(os.getcwd()[:studyPathInd+len(study)],'Analysis','Matlab','Behavior','csvfiles')
    behavData = pd.read_csv(op.join(behavPath, subj+'.csv'))
    behavData_half1 = behavData[0:300]
    behavData_half2 = behavData[300:]

    thisDeltaVal = np.array([
      np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==1]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==1])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==2]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==2])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==3]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==3])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==4]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==4])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==5]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==5])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==1]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==1])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==2]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==2])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==3]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==3])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==4]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==4])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==5]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==5])           
    ])
    
    color_thisDeltaVal = np.array([
      np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==1]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==1])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==2]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==2])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==3]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==3])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==4]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==4])
    , np.nanmean(behavData_half1.ColorValue[behavData_half1.Color==5]) - np.nanmean(behavData_half2.ColorValue[behavData_half2.Color==5])           
    ])
    
    
    face_thisDeltaVal = np.array([
      np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==1]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==1])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==2]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==2])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==3]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==3])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==4]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==4])
    , np.nanmean(behavData_half1.FaceValue[behavData_half1.Face==5]) - np.nanmean(behavData_half2.FaceValue[behavData_half2.Face==5])
    ])
    
    for t in range(0,60):
        decoderDeltaVal[(10*s):(10*s)+10,t] = thisDeltaVal   
        face_decoderDeltaVal[(5*s):(5*s)+5,t] = face_thisDeltaVal      
        color_decoderDeltaVal[(5*s):(5*s)+5,t] = color_thisDeltaVal        
        attr_subjID[(5*s):(5*s)+5,t] = s    
        subjID[(10*s):(10*s)+10,t] = s    
        
        
    #permute the labels, for use later on.
    for p in range(0,1000):
        decoderDeltaVal_permutations[(10*s):(10*s)+10,p] = np.random.permutation(thisDeltaVal) 
        face_decoderDeltaVal_permutations[(5*s):(5*s)+5,p] = np.random.permutation(face_thisDeltaVal) 
        color_decoderDeltaVal_permutations[(5*s):(5*s)+5,p] = np.random.permutation(color_thisDeltaVal)      
    

NameError: name 'os' is not defined

# 2. Conduct linear mixed effect models

## 2a. Value * Attribute Type

In [49]:
# Now iterate through each time-point, get coefficient and p-vals
betas_val = np.empty([60])
betas_val_SE = np.empty([60])
pvals_val = np.empty([60])

betas_Attr = np.empty([60])
betas_Attr_SE = np.empty([60])
pvals_Attr = np.zeros([60])

for t in range(0,60):
   
    d = {'deltaVal':decoderDeltaVal[:,t],'subjectID':subjID[:,t],'deltaDecoderAcc':decoderDeltaAcc[:,t],'Attr':faceOrcolor}
    df = pd.DataFrame(data=d)
    
    # Run the LMM
    lmm_inter = smf.mixedlm("deltaDecoderAcc ~ deltaVal * Attr", df, groups=df["subjectID"],
                        re_formula="~1").fit()
    betas_val[t]= lmm_inter.params[1]
    betas_val_SE[t]= lmm_inter.bse[1]
    pvals_val[t] = lmm_inter.pvalues[1]
    
    
    betas_Attr[t]= lmm_inter.params[3]
    betas_Attr_SE[t]= lmm_inter.bse[3]
    pvals_Attr[t] = lmm_inter.pvalues[3]
    

## 2b. abs(Value) * Attribute Type

In [ ]:
# Now iterate through each time-point, get coefficient and p-vals
abs_betas_val = np.empty([60])
abs_betas_val_SE = np.empty([60])
abs_pvals_val = np.empty([60])

abs_betas_Attr = np.empty([60])
abs_betas_Attr_SE = np.empty([60])
abs_pvals_Attr = np.zeros([60])

for t in range(0,60):
   
    d = {'deltaVal':np.abs(decoderDeltaVal[:,t]),'subjectID':subjID[:,t],'deltaDecoderAcc':decoderDeltaAcc[:,t],'Attr':faceOrcolor}
    df = pd.DataFrame(data=d)
    
    # Run the LMM
    lmm_inter = smf.mixedlm("deltaDecoderAcc ~ deltaVal * Attr", df, groups=df["subjectID"],
                        re_formula="~1").fit()
    abs_betas_val[t]= lmm_inter.params[1]
    abs_betas_val_SE[t]= lmm_inter.bse[1]
    abs_pvals_val[t] = lmm_inter.pvalues[1]
    
    
    abs_betas_Attr[t]= lmm_inter.params[3]
    abs_betas_Attr_SE[t]= lmm_inter.bse[3]
    abs_pvals_Attr[t] = lmm_inter.pvalues[3]

# 3. Compare to distribution from permutation testing (i.e., Step 5)

## 3a. Value * Attribute Type

In [2]:
# Setup some empty variables
val_permuted_corrected = []
xterm_permuted_corrected = []
val_permuted_count_for_values = np.zeros([1000,50])
xterm_permuted_count_for_values = np.zeros([1000,50])
val_final_values = np.zeros([1,50])
xterm_final_values = np.zeros([1,50])
permuted_val_pvals_final = np.zeros([1,60])
permuted_xterm_final =np.zeros([1,60])


# Load the permutation results
val_permuted_pvals = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AttrVal_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))
xterm_permuted_pvals = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AttrXterm_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))

# Go throguh the permutation results and use reverse-length encoding to identify significant clusters of activity
# that naturally occured because of noise.
for p in range(0,1000):
    values,counts = rle.encode(val_permuted_pvals[p,:]<.05)
    for i in range(0,len(values)):
        if values[i] == True:
            this_count = counts[i]
            val_permuted_count_for_values[p,this_count] =val_permuted_count_for_values[p,this_count] + 1           
    values,counts = rle.encode(xterm_permuted_pvals[p,:]<.05)
    for i in range(0,len(values)):
        if values[i] == True:
            this_count = counts[i]
            xterm_permuted_count_for_values[p,this_count] =xterm_permuted_count_for_values[p,this_count] + 1
            
            
# Clusters of a certain length should also count for smaller clusters. (I.e., finding a cluster length of 2 should also count for a length of 1)
for p in range(0,1000):
    for j in range(1,50):        
        if val_permuted_count_for_values[p,j] > 0:
            for v in range(0,j):
                val_permuted_count_for_values[p,v] = val_permuted_count_for_values[p,v] + val_permuted_count_for_values[p,j]
        if xterm_permuted_count_for_values[p,j] > 0:
            for v in range(0,j):
                xterm_permuted_count_for_values[p,v] = xterm_permuted_count_for_values[p,v] + xterm_permuted_count_for_values[p,j]

# Now, for runs of length 0 to 50, calculate the cluster adjusted p-value for a cluster length of i.
for i in range(0,50):
    val_final_values[0,i] = np.mean(val_permuted_count_for_values[:,i])
    xterm_final_values[0,i] = np.mean(xterm_permuted_count_for_values[:,i])

# Load the permuted coefficients.
val_permuted_coefs = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AttrVal_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))
xterm_permuted_coefs = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AttrXterm_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))


# Calculate a permutation-corrected p-value by comparing the real beta values against the permuted ones.
for t in range(0,60):
    permuted_val_pvals_final[0,t] =(sum(betas_val[t] < val_permuted_coefs[:,t])/1000)
    permuted_xterm_final[0,t] =(sum(betas_Attr[t] < xterm_permuted_coefs[:,t])/1000)


# Search these permutated-corrected p-values for clusters of significant activity, using reverse length encoding
# When a significant cluster is found, it will use the cluster-corrected p-value. 
val_pvals_corrected = []
value, count = rle.encode(permuted_val_pvals_final[0] < .05)
for thisValue in range(0,len(value)):
    if value[thisValue] == False:
        for i in range(0,count[thisValue]):
            val_pvals_corrected.append(2)
    elif value[thisValue] == True:
        thisCount = count[thisValue]
        for i in range(0,count[thisValue]):
            val_pvals_corrected.append(val_final_values[0,thisCount])
            
xterm_pvals_corrected = []
value, count = rle.encode(permuted_xterm_final[0] < .05)
for thisValue in range(0,len(value)):
    if value[thisValue] == False:
        for i in range(0,count[thisValue]):
            xterm_pvals_corrected.append(2)
    elif value[thisValue] == True:
        thisCount = count[thisValue]
        for i in range(0,count[thisValue]):
            xterm_pvals_corrected.append(xterm_final_values[0,thisCount])          
            

    
time = np.arange(-200,1000,20)
# Identify significant cluster-corrected p-values and translate them into a plot.
sigLine_Val_cc   = np.zeros(time.shape)
sigLine_Val_cc   = np.zeros(time.shape)
sigLine_Val_cc[np.array(val_pvals_corrected)<.05]=.038
sigLine_Val_cc[np.array(val_pvals_corrected)>=.05]=-.1

sigLine_Xterm_cc   = np.zeros(time.shape)
sigLine_Xterm_cc[np.array(xterm_pvals_corrected)<.05]=.038
sigLine_Xterm_cc[np.array(xterm_pvals_corrected)>=.05]=-.1

NameError: name 'np' is not defined

## 3b. abs(Value) * Attribute Type

In [ ]:
abs_val_permuted_pvals = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrVal_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))
abs_xterm_permuted_pvals = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrXterm_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))


abs_val_permuted_corrected = []
abs_xterm_permuted_corrected = []


abs_val_permuted_count_for_values = np.zeros([1000,50])
abs_xterm_permuted_count_for_values = np.zeros([1000,50])

abs_val_final_values = np.zeros([1,50])
abs_xterm_final_values = np.zeros([1,50])

for p in range(0,1000):


    values,counts = rle.encode(abs_val_permuted_pvals[p,:]<.05)
    for i in range(0,len(values)):
        if values[i] == True:
            this_count = counts[i]
            abs_val_permuted_count_for_values[p,this_count] =abs_val_permuted_count_for_values[p,this_count] + 1
            
    values,counts = rle.encode(abs_xterm_permuted_pvals[p,:]<.05)
    for i in range(0,len(values)):
        if values[i] == True:
            this_count = counts[i]
            abs_xterm_permuted_count_for_values[p,this_count] =abs_xterm_permuted_count_for_values[p,this_count] + 1
            

            

# New as of March 17th, 2025 - now includes later values (for smaller ones.)
for p in range(0,1000):
    for j in range(1,50):        
        if abs_val_permuted_count_for_values[p,j] > 0:
            for v in range(0,j):
                abs_val_permuted_count_for_values[p,v] = abs_val_permuted_count_for_values[p,v] + abs_val_permuted_count_for_values[p,j]
        if abs_xterm_permuted_count_for_values[p,j] > 0:
            for v in range(0,j):
                abs_xterm_permuted_count_for_values[p,v] = abs_xterm_permuted_count_for_values[p,v] + abs_xterm_permuted_count_for_values[p,j]

            
for i in range(0,50):
    abs_val_final_values[0,i] = np.mean(abs_val_permuted_count_for_values[:,i])
    abs_xterm_final_values[0,i] = np.mean(abs_xterm_permuted_count_for_values[:,i])
  
    
abs_val_permuted_coefs = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrVal_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))
abs_xterm_permuted_coefs = np.load(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrXterm_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'))


abs_permuted_val_pvals_final = np.zeros([1,60])
abs_permuted_xterm_final =np.zeros([1,60])

for t in range(0,60):
    abs_permuted_val_pvals_final[0,t] =(sum(abs_betas_val[t] < abs_val_permuted_coefs[:,t])/1000)
    abs_permuted_xterm_final[0,t] =(sum(abs_betas_Attr[t] < abs_xterm_permuted_coefs[:,t])/1000)




abs_val_pvals_corrected = []
value, count = rle.encode(abs_permuted_val_pvals_final[0] < .05)
for thisValue in range(0,len(value)):
    if value[thisValue] == False:
        for i in range(0,count[thisValue]):
            abs_val_pvals_corrected.append(2)
    elif value[thisValue] == True:
        thisCount = count[thisValue]
        for i in range(0,count[thisValue]):
            abs_val_pvals_corrected.append(abs_val_final_values[0,thisCount])
            
abs_xterm_pvals_corrected = []
value, count = rle.encode(abs_permuted_xterm_final[0] < .05)
for thisValue in range(0,len(value)):
    if value[thisValue] == False:
        for i in range(0,count[thisValue]):
            abs_xterm_pvals_corrected.append(2)
    elif value[thisValue] == True:
        thisCount = count[thisValue]
        for i in range(0,count[thisValue]):
            abs_xterm_pvals_corrected.append(abs_xterm_final_values[0,thisCount])          
            

    
time = np.arange(-200,1000,20)

abs_sigLine_Val_cc   = np.zeros(time.shape)
abs_sigLine_Val_cc   = np.zeros(time.shape)
abs_sigLine_Val_cc[np.array(abs_val_pvals_corrected)<.05]=.038
abs_sigLine_Val_cc[np.array(abs_val_pvals_corrected)>=.05]=-.1


abs_sigLine_Xterm_cc   = np.zeros(time.shape)
abs_sigLine_Xterm_cc[np.array(abs_xterm_pvals_corrected)<.05]=.038
abs_sigLine_Xterm_cc[np.array(abs_xterm_pvals_corrected)>=.05]=-.1


# 4. Plot the resulting effects.

## 4a. Value * Attribute Type

In [87]:
## Value Across
%matplotlib qt
plt.plot(time,betas_val,c='blue')
plt.fill_between(time,betas_val-betas_val_SE,betas_val+betas_val_SE,alpha=.2)
plt.axis((-200,1000,-.04,.04))
plt.xlabel(xlabel='time')
plt.title('Value')
plt.ylabel('Coefficient from LMM')
plt.plot(time,sigLine_Val_cc,marker='s',  markersize=2,c='red',label='cluster-permuted pvals')
plt.savefig('Decoding_by_Val_Value_Ribbon_Testing_Testing_20250317_reref_20ms-upto1000ms-doublecheck2-newSVMs_attrFix.eps', format='eps')


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [ ]:
## Value Xterm
%matplotlib qt
plt.plot(time,betas_Attr,c='blue')
plt.fill_between(time,betas_Attr-betas_Attr_SE,betas_Attr+betas_Attr_SE,alpha=.2)
plt.axis((-200,1000,-.04,.040))
plt.xlabel(xlabel='time')
plt.title('Xterm')
plt.ylabel('Coefficient from LMM')
plt.plot(time,sigLine_Xterm_cc,marker='s',  markersize=2,c='red',label='cluster-permuted pvals')
plt.savefig('Decoding_by_Val_Xterm_Ribbon_Testing_Testing_20250317_reref_20ms-upto1000ms-doublecheck2-newSVMs_attrFix.eps', format='eps')


## 4b. abs(Value) * Attribute Type

In [ ]:
## Value Across
%matplotlib qt
plt.plot(time,abs_betas_val,c='blue')
plt.fill_between(time,abs_betas_val-abs_betas_val_SE,abs_betas_val+abs_betas_val_SE,alpha=.2)
plt.axis((-200,1000,-.04,.04))
plt.xlabel(xlabel='time')
plt.title('Value AbslVal')
plt.ylabel('Coefficient from LMM')
plt.plot(time,abs_sigLine_Val_cc,marker='s',  markersize=2,c='red',label='cluster-permuted pvals')
plt.savefig('Decoding_by_Val_AbsVal_Value_Ribbon_Testing_Testing_20250317_reref_20ms-upto1000ms-doublecheck2-newSVMs_attrFix.eps', format='eps')


In [96]:
## Value Xterm
%matplotlib qt
plt.plot(time,abs_betas_Attr,c='blue')
plt.fill_between(time,abs_betas_Attr-abs_betas_Attr_SE,abs_betas_Attr+abs_betas_Attr_SE,alpha=.2)
plt.axis((-200,1000,-.04,.040))
plt.xlabel(xlabel='time')
plt.title('Xterm AbsVal')
plt.ylabel('Coefficient from LMM')
plt.plot(time,abs_sigLine_Xterm_cc,marker='s',  markersize=2,c='red',label='cluster-permuted pvals')
plt.savefig('Decoding_by_Val_AbsVal_Xterm_Ribbon_Testing_Testing_20250317_reref_20ms-upto1000ms-doublecheck2-newSVMs_attrFix.eps', format='eps')


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# 5. Permutation testing and saving output (only run once)

## 5a. Value * Attribute Type

In [2]:
permuted_AttrVal_coefs = np.empty([1000,60])
permuted_AttrXterm_coefs = np.empty([1000,60])

permuted_AttrVal_pvals = np.empty([1000,60])
permuted_AttrXterm_pvals = np.empty([1000,60])


for t in range(0,60):
    for p in range(0,1000):      
        
        # Overall + Attr
        d = {'deltaVal':decoderDeltaVal_permutations[:,p],'subjectID':subjID[:,t],'deltaDecoderAcc':decoderDeltaAcc[:,t],'Attr':faceOrcolor}
        df = pd.DataFrame(data=d)
        lmm_inter = smf.mixedlm("deltaDecoderAcc ~ deltaVal * Attr", df, groups=df["subjectID"],
                        re_formula="~1").fit()
        permuted_AttrVal_coefs[p,t] =  (lmm_inter.params[1])    
        permuted_AttrVal_pvals[p,t] =  (lmm_inter.pvalues[1]) 
        
        permuted_AttrXterm_coefs[p,t] =  (lmm_inter.params[3])    
        permuted_AttrXterm_pvals[p,t] =  (lmm_inter.pvalues[3]) 
       
           
    print("time bin finished: " + str(t))

np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AttrVal_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),permuted_AttrVal_coefs)
np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AttrXterm_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),permuted_AttrXterm_coefs)

np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AttrVal_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),permuted_AttrVal_pvals)
np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AttrXterm_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),permuted_AttrXterm_pvals)


NameError: name 'np' is not defined

##  5b. abs(Value) * Attribute Type

In [64]:
abs_permuted_AttrVal_coefs = np.empty([1000,60])
abs_permuted_AttrXterm_coefs = np.empty([1000,60])

abs_permuted_AttrVal_pvals = np.empty([1000,60])
abs_permuted_AttrXterm_pvals = np.empty([1000,60])


for t in range(0,60):
    for p in range(0,1000):      
        
        # Overall + Attr
        d = {'deltaVal':np.abs(decoderDeltaVal_permutations[:,p]),'subjectID':subjID[:,t],'deltaDecoderAcc':decoderDeltaAcc[:,t],'Attr':faceOrcolor}
        df = pd.DataFrame(data=d)
        lmm_inter = smf.mixedlm("deltaDecoderAcc ~ deltaVal * Attr", df, groups=df["subjectID"],
                        re_formula="~1").fit()
        abs_permuted_AttrVal_coefs[p,t] =  (lmm_inter.params[1])    
        abs_permuted_AttrVal_pvals[p,t] =  (lmm_inter.pvalues[1]) 
        
        abs_permuted_AttrXterm_coefs[p,t] =  (lmm_inter.params[3])    
        abs_permuted_AttrXterm_pvals[p,t] =  (lmm_inter.pvalues[3]) 
       
           
    print("time bin finished: " + str(t))

np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrVal_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),abs_permuted_AttrVal_coefs)
np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrXterm_Coefs_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),abs_permuted_AttrXterm_coefs)

np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrVal_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),abs_permuted_AttrVal_pvals)
np.save(op.join('E:\Bachman\MADeEEG1','Permuted_AbsVal_AttrXterm_Pvals_reref_20ms-upto1000ms-doublecheck-newSVMs_attrFix.npy'),abs_permuted_AttrXterm_pvals)


time bin finished: 0
time bin finished: 1
time bin finished: 2
time bin finished: 3
time bin finished: 4
time bin finished: 5
time bin finished: 6
time bin finished: 7
time bin finished: 8
time bin finished: 9
time bin finished: 10
time bin finished: 11
time bin finished: 12
time bin finished: 13
time bin finished: 14
time bin finished: 15
time bin finished: 16
time bin finished: 17
time bin finished: 18
time bin finished: 19
time bin finished: 20
time bin finished: 21
time bin finished: 22
time bin finished: 23
time bin finished: 24
time bin finished: 25
time bin finished: 26
time bin finished: 27
time bin finished: 28
time bin finished: 29
time bin finished: 30
time bin finished: 31
time bin finished: 32
time bin finished: 33
time bin finished: 34
time bin finished: 35
time bin finished: 36
time bin finished: 37
time bin finished: 38
time bin finished: 39
time bin finished: 40
time bin finished: 41
time bin finished: 42
time bin finished: 43
time bin finished: 44
time bin finished: 4